## The try Statement

### `try/except`

In [1]:
try: 
    1/0
    print('not executed') 
except ZeroDivisionError:
    print('caught divide-by-0 attempt')

caught divide-by-0 attempt


Let's consider more interesting example.

In [2]:
try:
    with open('example.txt', 'r') as f:
        # Attempt to read the file
        content = f.read()
        print(content)
        
        # Simulate a potential division by zero error
        result = 10 / 0

except FileNotFoundError:
    print("Error: The file 'example.txt' was not found.")
except PermissionError:
    print("Error: You do not have permission to read 'example.txt'.")
except ZeroDivisionError:
    print("Error: A division by zero occurred.")

Error: The file 'example.txt' was not found.


Let's now add this file.

In [6]:
try:
    with open('example.txt', 'a') as f:
        # Attempt to read the file
        content = f.read()
        print(content)
        
        # Simulate a potential division by zero error
        result = 10 / 0

except FileNotFoundError:
    print("Error: The file 'example.txt' was not found.")
except PermissionError:
    print("Error: You do not have permission to read 'example.txt'.")
except ZeroDivisionError:
    print("Error: A division by zero occurred.")

Error: You do not have permission to read 'example.txt'.


Let's now try to change permissions.

In [8]:
try:
    with open('example.txt', 'a') as f:
        # Attempt to read the file
        file.write('\nThis is a new line of text.')
        # content = f.read()
        # print(content)
        
        # Simulate a potential division by zero error
        result = 10 / 0

except FileNotFoundError:
    print("Error: The file 'example.txt' was not found.")
except PermissionError:
    print("Error: You do not have permission to write into 'example.txt'.")
except ZeroDivisionError:
    print("Error: A division by zero occurred.")

Error: You do not have permission to write into 'example.txt'.


It seems we can have `try` without `except`. In this case it must be with `finally`. It's not that easy to find a user case for that. But it actually exists.

> **Explanation**
> - Lock Acquisition: The lock is acquired before entering the critical section.
> - `try` Block: Contains code that needs synchronization. If an exception occurs here, the finally block will still execute.
> - `finally` Block: Ensures that the lock is released, preventing deadlocks even if an exception occurs in the try block.
>
> **Why Use try/finally Here?**
> - Guarantee Cleanup: Ensures that the lock is released no matter what, which is crucial for preventing deadlocks in multithreaded applications.
> - Separation of Concerns: The `try/finally` construct focuses on cleanup, while exception handling can be managed elsewhere if needed.
>
> In summary, while `try/finally` might not be as commonly used as `try/except`, it plays an important role in scenarios where resource cleanup is critical, such as with locks, network connections, or other resources that require explicit release or closure.

In [9]:
import threading

lock = threading.Lock()

def critical_section():
    lock.acquire()
    try:
        # Perform operations that require synchronization
        print("Critical section is being executed.")
    finally:
        # Ensure the lock is released
        lock.release()
        print("Lock has been released.")

# Simulating the function call
critical_section()

Critical section is being executed.
Lock has been released.


In Python we'd rather use `with` for this purpose.

In [10]:
import threading

lock = threading.Lock()

def critical_section():
    with lock:
        # Perform operations that require synchronization
        print("Critical section is being executed.")

# Simulating the function call
critical_section()

Critical section is being executed.


In [11]:
lock is None

False

In [12]:
if lock.locked():
    print("The lock is currently held.")
else:
    print("The lock is not held.")

The lock is not held.


#### `else`

> The optional else clause of try/except executes only when the try clause termi‐ nates normally.

In [13]:
value = None

In [14]:
print(repr(value), 'is ', end=' ') 
try:
    value + 0 
except TypeError:
    # not a number, maybe a string...?
    try:
        value + ''
    except TypeError:
        print('neither a number nor a string')
    else:
        print('some kind of string')
else:
    print('some kind of number')

None is  neither a number nor a string


In [15]:
value = 'hello'

In [16]:
print(repr(value), 'is ', end=' ') 
try:
    value + 0 
except TypeError:
    # not a number, maybe a string...?
    try:
        value + ''
    except TypeError:
        print('neither a number nor a string')
    else:
        print('some kind of string')
else:
    print('some kind of number')

'hello' is  some kind of string


#### Exception propagation

It turns out it has many nuances as well.  What we know that it halts execution as soon as an exception happened. It searches for the *first* `except` clause it matches and executes its handler. It checks no further clauses. That's why we need to order exception clauses from more specific to less specific.

Here's an example that is new for me.

In [19]:
try: 
    try:
        1/0

    except:
        print('caught an exception')
except ZeroDivisionError:
    print('caught divide-by-0 attempt')

caught an exception


> In this case, it does not matter that the handler established by the clause `except ZeroDivisionError:` in the outer `try` clause is more specific than the catch-all except: in the inner try clause. The outer try does not enter into the picture: the exception doesn’t propagate out of the inner `try`.

## The `raise` statement

`raise [expression [from exception]]`

It turns out that even with `raise` there are many cases that I'm not aware of:
- First of all we may raise without `expression`. It's possible only by an exception handler.
- It's also possible to wrap one exception into another using `from exception`.

## The `with` Statement and Context Managers